In [1]:
# getting the dataset for training 
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-15 11:45:45--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2025-05-15 11:45:46 (7.91 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
# read and inspeect the text file
with open('input.txt', 'r') as f:
    text = f.read()

In [3]:
# exploring the txt file

# looking at first 1000 characters
print(text[:1000], '\n')

#number of characters in the dataset
print(f"length of the dataset: {len(text)}")

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# vocabulary and the size of vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"vocabulary: {''.join(chars)}")
print(f"size of vocabulary: {vocab_size}")

vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
size of vocabulary: 65


In [5]:
# tokenizing the text; ie convert the raw strings to some sequences of integers

# creating a mapping of characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda st: [stoi[x] for x in st] # function of the string which takes the string and returns a list of integers
decode = lambda l: ''.join([itos[x] for x in l]) # function of the lost of integers where it takes each elements of the list and returns a character and joins them together

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
# encoding the entire dataset and storing it as a torch tensor
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# splitting the data into train and validation sets
n = int(0.9 * len(data)) # first 90% will be train, rest 10% will be validation set
train_data = data[:n]
val_data = data[n:]
# validation set will help us understand how much our model is overfitting and memorising 

In [10]:
# the entire text is never fed into the transformer in one go as it is computationally prohibitive
# chunking the dataset and sample these chunks into the transformer